In [127]:
import matplotlib.pyplot as plt
import math
import time
import random
import copy

import numpy as np
import copy
import torch
from scipy import signal
from scipy import misc

In [156]:
class Agent_Conv:
    def __init__(self, num_input, layers, num_output):
        assert type(layers)==type([]), "Error with layers, give array of the number of layers"
        self.num_input = num_input  #set input number
        self.num_output = num_output #set ooutput number
        self.hidden=[]
        last=num_input
        self.num_genes=0
        for layer in layers:
            self.hidden.append(layer)
            self.num_genes+=(last * layer)
            last=layer
        self.num_genes +=(self.hidden[-1]*num_output)+num_output
        self.weights = None
        self.hidden_weights=None
        self.bias = None
        print("Auto",self.num_genes)
    def set_genes(self, gene):
        weight_idxs = self.num_input * self.hidden[0] #size of weights to hidden
        current=weight_idxs
        weights_idxs=[current] #start with end of last
        for i in range(len(self.hidden)-1):
            current+=self.hidden[i]*self.hidden[i+1] #calculate next idx for each layer
            weights_idxs.append(current)
        bias_idxs=None
        weights_idxs.append(self.hidden[-1] * self.num_output + weights_idxs[-1]) #add last layer heading to output
        bias_idxs = weights_idxs[-1]+ self.num_output #sizes of biases
        w = gene[0 : weight_idxs].reshape(self.hidden[0], self.num_input)   #merge genes
        ws=[]
        for i in range(len(self.hidden)-1):
            ws.append(gene[weights_idxs[i] : weights_idxs[i+1]].reshape(self.hidden[i+1], self.hidden[i]))
        ws.append(gene[weights_idxs[-2] : weights_idxs[-1]].reshape(self.num_output, self.hidden[-1]))
        b = gene[weights_idxs[-1]: bias_idxs].reshape(self.num_output,) #merge genes

        self.weights = torch.from_numpy(w) #assign weights
        self.hidden_weights=[]
        for w in ws:
            self.hidden_weights.append(torch.from_numpy(w))
        self.bias = torch.from_numpy(b) #assign biases

    def forward(self, x):
        #create conv layer
        scharr = np.array([[ -3-3, 0-10,  +3 -3],
                   [-10+0, 0+ 0, +10 +0],
                   [ -3+3, 0+10,  +3 +3]]) # Gx + j*Gy
        x= signal.convolve2d(x, scharr, boundary='symm', mode='same')   
        x=x.flatten()
        x=x[:,np.newaxis]      
        x = torch.tensor(np.dot(self.weights.float(),x).flatten()).float()
        #run through forward layers
        x=x[:,np.newaxis]
        for i in range(len(self.hidden_weights)-1):
            x = torch.mm(x.T,self.hidden_weights[i].T.float()) #second layer
        return torch.mm(x,self.hidden_weights[-1].T.float()) + self.bias #third layer
    
    def get_action(self, x):
        vectors=[(1,1),(1,0),(0,1),(-1,-1),(-1,0),(0,-1),(-1,1),(1,-1)] #possible moves
        arr=list(self.forward(x)[0])
        ind=np.argmax(arr)
        return vectors[ind]

In [ ]:
from scipy import signal
from scipy import misc
ascent = misc.ascent()
scharr = np.array([[ -3-3, 0-10,  +3 -3],
                   [-10+0, 0+ 0, +10 +0],
                   [ -3+3, 0+10,  +3 +3]]) # Gx + j*Gy
ascent=np.random.rand(5,5)
grad = signal.convolve2d(ascent, scharr, boundary='symm', mode='same')
grad

In [160]:
whegBot=Agent_Conv(5*5,[10,10],10) #define the agent


input_=np.random.rand(5,5)
output_=np.random.rand(10)

whegBot.set_genes(np.random.normal(0, 0.5, (whegBot.num_genes)))
print(input_)

whegBot.get_action(input_)

Auto 460
[[0.6407363  0.82055014 0.82814506 0.15915161 0.59689979]
 [0.19818143 0.91646541 0.45992971 0.34093896 0.3269904 ]
 [0.4780956  0.84146013 0.26159009 0.88285173 0.16603833]
 [0.39397358 0.41748906 0.10699987 0.90084984 0.82119487]
 [0.86843626 0.73591188 0.85115252 0.43951189 0.32143432]]


(0, 1)